In [2]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class ImageProcessingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processing Demo")
        
        # Biến lưu trữ ảnh
        self.original_image = None
        self.processed_image = None
        
        self.setup_ui()
    
    def setup_ui(self):
        # Frame chính
        main_frame = ttk.Frame(self.root)
        main_frame.pack(expand=True, fill='both', padx=5, pady=5)
        
        # Frame điều khiển bên trái
        control_frame = ttk.LabelFrame(main_frame, text="Controls")
        control_frame.pack(side='left', fill='y', padx=5)
        
        # Nút tải ảnh
        ttk.Button(control_frame, text="Load Image", command=self.load_image).pack(pady=5)
        
        # Nút Reset
        ttk.Button(control_frame, text="Reset", command=self.reset_image).pack(pady=5)
        
        # Frame cho color balance
        color_frame = ttk.LabelFrame(control_frame, text="Color Balance")
        color_frame.pack(fill='x', padx=5, pady=5)
        
        # Sliders cho color balance
        self.red_var = tk.DoubleVar(value=1.0)
        self.green_var = tk.DoubleVar(value=1.0)
        self.blue_var = tk.DoubleVar(value=1.0)
        
        ttk.Label(color_frame, text="Red:").pack()
        ttk.Scale(color_frame, from_=0.0, to=2.0, variable=self.red_var, 
                 command=self.update_color_balance).pack()
        
        ttk.Label(color_frame, text="Green:").pack()
        ttk.Scale(color_frame, from_=0.0, to=2.0, variable=self.green_var,
                 command=self.update_color_balance).pack()
        
        ttk.Label(color_frame, text="Blue:").pack()
        ttk.Scale(color_frame, from_=0.0, to=2.0, variable=self.blue_var,
                 command=self.update_color_balance).pack()
        
        # Buttons cho các chức năng khác
        ttk.Button(control_frame, text="Show Histogram", 
                  command=self.show_histogram).pack(pady=5)
        ttk.Button(control_frame, text="Histogram Equalization", 
                  command=self.histogram_equalization).pack(pady=5)
        
        # Frame cho filters
        filter_frame = ttk.LabelFrame(control_frame, text="Filters")
        filter_frame.pack(fill='x', padx=5, pady=5)
        
        ttk.Button(filter_frame, text="Median Filter", 
                  command=self.apply_median_filter).pack(pady=5)
        ttk.Button(filter_frame, text="Mean Filter", 
                  command=self.apply_mean_filter).pack(pady=5)
        ttk.Button(filter_frame, text="Gaussian Smoothing", 
                  command=self.apply_gaussian_smoothing).pack(pady=5)
        
        # Frame hiển thị ảnh
        image_frame = ttk.Frame(main_frame)
        image_frame.pack(side='right', expand=True, fill='both')
        
        self.image_label = ttk.Label(image_frame)
        self.image_label.pack(expand=True)
    
    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.original_image = cv2.imread(file_path)
            self.processed_image = self.original_image.copy()
            self.reset_color_balance()  # Reset color balance when a new image is loaded
            self.display_image(self.processed_image)
    
    def reset_image(self):
        """Khôi phục lại ảnh gốc và các giá trị ban đầu."""
        self.processed_image = self.original_image.copy() if self.original_image is not None else None
        self.reset_color_balance()  # Reset color balance
        self.display_image(self.processed_image)
    
    def reset_color_balance(self):
        """Đặt lại giá trị của các sliders về mặc định."""
        self.red_var.set(1.0)
        self.green_var.set(1.0)
        self.blue_var.set(1.0)
    
    def display_image(self, image):
        if image is not None:
            # Chuyển từ BGR sang RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Chuyển sang định dạng PIL
            image_pil = Image.fromarray(image_rgb)
            # Tạo PhotoImage
            photo = ImageTk.PhotoImage(image_pil)
            # Hiển thị
            self.image_label.configure(image=photo)
            self.image_label.image = photo
    
    def update_color_balance(self, _=None):
        if self.original_image is not None:
            # Lấy giá trị từ sliders
            r, g, b = self.red_var.get(), self.green_var.get(), self.blue_var.get()
            # Áp dụng color balance
            self.processed_image = self.original_image.copy()
            self.processed_image[:,:,2] = np.clip(self.processed_image[:,:,2] * r, 0, 255)
            self.processed_image[:,:,1] = np.clip(self.processed_image[:,:,1] * g, 0, 255)
            self.processed_image[:,:,0] = np.clip(self.processed_image[:,:,0] * b, 0, 255)
            self.display_image(self.processed_image)
    
    def show_histogram(self):
        if self.processed_image is not None:
            # Tạo cửa sổ mới cho histogram
            hist_window = tk.Toplevel(self.root)
            hist_window.title("Image Histogram")
            
            # Tạo figure cho matplotlib
            fig, ax = plt.subplots(figsize=(6, 4))
            
            # Tính histogram cho mỗi kênh màu
            colors = ('b', 'g', 'r')
            for i, color in enumerate(colors):
                hist = cv2.calcHist([self.processed_image], [i], None, [256], [0, 256])
                ax.plot(hist, color=color)
            
            ax.set_xlim([0, 256])
            ax.set_title('Color Histogram')
            
            # Nhúng biểu đồ vào tkinter
            canvas = FigureCanvasTkAgg(fig, master=hist_window)
            canvas.draw()
            canvas.get_tk_widget().pack()
    
    def histogram_equalization(self):
        if self.processed_image is not None:
            # Chuyển sang grayscale
            gray = cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2GRAY)
            # Thực hiện cân bằng histogram
            equalized = cv2.equalizeHist(gray)
            # Chuyển lại sang BGR
            self.processed_image = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
            self.display_image(self.processed_image)
    
    def apply_median_filter(self):
        if self.processed_image is not None:
            self.processed_image = cv2.medianBlur(self.processed_image, 5)
            self.display_image(self.processed_image)
    
    def apply_mean_filter(self):
        if self.processed_image is not None:
            self.processed_image = cv2.blur(self.processed_image, (5,5))
            self.display_image(self.processed_image)
    
    def apply_gaussian_smoothing(self):
        if self.processed_image is not None:
            self.processed_image = cv2.GaussianBlur(self.processed_image, (5,5), 0)
            self.display_image(self.processed_image)

if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessingApp(root)
    root.mainloop()
